In [ ]:
import os
import numpy as np
import codecs
import json
from datetime import datetime, date, timedelta
import mysql.connector
from mysql.connector import errorcode

DB_NAME = "TaxiTest"
config = {
        'user': 'root',
        'password': '9090',
        'host': '127.0.0.1',
        'database': 'taxitest',
        'raise_on_warnings': True,
        'use_pure': False
}

cnx = mysql.connector.connect(**config)
cnx.autocommit = True
cursor = cnx.cursor()


In [ ]:
query = "SHOW DATABASES"
cursor.execute(query)
rows = cursor.fetchall()
print("Total number of databases:", cursor.rowcount)
for row in rows:
    print(row)

In [ ]:
# DESCRIBE taxitest.order;

cursor.execute("USE " + DB_NAME)
cursor.execute("SHOW Tables")
tables = cursor.fetchall()


for table in tables:
    print("Table:", str(table[0]))
    
    cursor.execute("DESCRIBE %s.%s" % (DB_NAME, table[0]))
    columns_attributes = cursor.fetchall()
    
    for column_attributes_tuple in columns_attributes:
        column_attributes = list(column_attributes_tuple)
        column_attributes[1] = str(column_attributes[1])[2:-1]
        print(', '.join([str(a) for a in column_attributes]))
    
    print("\n")
    


In [ ]:
cursor.close()
cnx.close()

# Start

In [ ]:
# Getting streets ordered list

file = "CrawledStreets.txt"
streets = set()
with codecs.open(file, 'r', encoding='utf-8') as f:
    for line in f:
        streets.add(line.split(' ')[0])

# Choose unique
all_streets = sorted(list(streets))

streets = []
for street in all_streets:
    if len(street) >= 5:
        streets.append(street)

with open('Streets.json', 'w') as outfile:
    json.dump(streets, outfile)

In [ ]:
# Getting InsuranceCompanies list and saving in json 
file = "InsuranceCompanies\Companies.txt"

names = []
with codecs.open(file, 'r', encoding='utf-8') as f:
    for line in f:
        names.append(line[:-2])
        
with open('InsuranceCompanies\Companies.json', 'w') as outfile:
    json.dump(names, outfile)


# BASIC FUNCTIONS FOR ALL TABLES

In [ ]:
def get_table_review(table, only_columns_names = False):
    columns_names = []
    print("Reviewing ", table)
    q = "DESCRIBE " + table
    cursor.execute(q)
    columns_attributes = cursor.fetchall()
    for column_attributes_tuple in columns_attributes:
        column_attributes = list(column_attributes_tuple)
        column_attributes[1] = str(column_attributes[1])[2:-1]
        columns_names.append(column_attributes[0])
        if not only_columns_names:
            print(', '.join([str(a) for a in column_attributes]))
    
    joined = ', '.join(columns_names)
    joined_values = ', '.join('%s' for c in columns_names)
    print("\nColumns: (", joined, ")")
    insert_string = "query = \"INSERT INTO %s (%s) VALUES (%s)\"" % (table, joined, joined_values)
    print(insert_string)
    
def count_rows(table):
    q = "SELECT COUNT(*) FROM " + table
    cursor.execute(q)
    log = cursor.fetchall()
    print("Total number of rows ", log)
    return int(log[0][0])

def delete_all_rows(table):
    q = "DELETE FROM " + table
    cursor.execute(q) 
    
def select_rows(table, q):
    cursor.execute(q)
    return cursor.fetchall()

def insert_many_rows(table, q, data):
    try:
        cursor.executemany(q, data)
        log = cursor.fetchall()
        print("Result: " + log)
    except mysql.connector.Error as err:
        print(err.msg)

## Table `insurancecompany`

In [ ]:
table = DB_NAME + "." + "insurancecompany"

In [ ]:
get_table_review(table)

In [ ]:
count_rows(table)

In [ ]:
delete_all_rows(table)

In [ ]:
# Data generation

with open('InsuranceCompanies/Companies.json') as f:
    names = json.load(f)

total_number = len(names)
print("len(names): ", len(names))
idInsuranceCompany = np.arange(total_number)

with open('Streets.json') as f:
    streets = json.load(f)
    streets = np.random.choice(streets, total_number, replace=False)

query = "INSERT INTO insurancecompany (idInsuranceCompany, Address, Phone, Mail, Name) VALUES (%s, %s, %s, %s, %s)"
data = []

for i in range(total_number):
    idInsuranceCompany = i + 1
    address = "Kazan, " + streets[i] + ", " + str(np.random.randint(1, 200))
    phone = '+7(843)' + '{:07}'.format(np.random.randint(0, 9999999))
    name = names[i]
    mail = name.lower().replace(" ", "") + '@gmail.com'
    data.append((idInsuranceCompany, address, phone, mail, name))

print("Q: ", query)
print("D: ", data[0])

insert_many_rows(table, query, data)

## Table `insurance`

*** Changes ***
~~~~python
# Modify type
cursor.execute("ALTER TABLE " + table + " MODIFY InsuranceStartDate DATE")
cursor.execute("ALTER TABLE " + table + " MODIFY InsuranceEndDate DATE")

# Change order
cursor.execute("ALTER TABLE " + table + " CHANGE COLUMN InsuranceEndDate InsuranceEndDate Date AFTER InsuranceStartDate")
~~~~

In [ ]:
table = DB_NAME + "." + "insurance"

In [ ]:
get_table_review(table)

In [ ]:
count_rows(table)

In [ ]:
delete_all_rows(table)

In [ ]:
# Data generation

query = "INSERT INTO TaxiTest.insurance (idInsurance, InsCompany_idInsCompany, InsuranceEndDate, InsuranceStartDate) VALUES (%s , %s , %s , %s )"
data = []

total_number = 100

for i in range(total_number):
    idInsurance = i + 1 
    InsCompany_idInsCompany = np.random.randint(1, 30)
    InsuranceStartDate = date(2014 + np.random.randint(-3, 3), np.random.randint(1, 10), np.random.randint(1, 25))
    InsuranceEndDate = InsuranceStartDate + timedelta(days=25*int(np.random.choice(np.array([6, 8, 12]))))
    data.append((idInsurance, InsCompany_idInsCompany, InsuranceStartDate, InsuranceEndDate))
    #print(data[i])
    
print("Q: ", query)
print("D: ", data[0])

insert_many_rows(table, query, data)

## Table `car`

*** Changes ***
~~~~python
# Modify type
cursor.execute("ALTER TABLE " + table + " MODIFY ChildSeat INT")
~~~~


In [ ]:
# Load the data from stored text file
# 1. Make json file
file = "Car/MarkModel.txt"
cars = set()
with codecs.open(file, 'r', encoding='utf-8') as f:
    for line in f:
        cars.add(line[:-2])

# Choose unique
cars = sorted(list(cars))
with open('Car/Cars.json', 'w') as outfile:
    json.dump(cars, outfile)


In [ ]:
table = DB_NAME + "." + "car"
get_table_review(table)
count_rows(table)

In [ ]:
import string
import random

# Data generation
query = "INSERT INTO TaxiTest.car (idCar, Lisense, Mark, Model, Insurance_idInsurance, ChildSeat) VALUES (%s, %s, %s, %s, %s, %s)"
data = []

total_number = 100
# Get insurances (better in random order), we have 100 insurances -> total_number = 100 of cars available
# Warning: np.arange(a) return from 0 to a - 1
# Writing Insurance_idInsurance which doesn't exist in insurance is restricted - QUERY will not be executed
insurances = np.arange(1, total_number + 1)
np.random.shuffle(insurances)

# Load cars
with open("Car/Cars.json") as f:
    cars = json.load(f)

# Generate UNIQUE car licenses
licenses = set()
alphanums = string.ascii_uppercase + string.digits
while len(licenses) != 100:
    licenses.add(''.join(random.choice(alphanums) for _ in range(7)))
licenses = list(licenses)

for i in range(total_number):
    idCar = i + 1 
    Lisense = licenses[i]
    (Mark, Model) = np.random.choice(cars).split(' ')
    # Failed processing format-parameters; Python 'int32' cannot be converted to a MySQL type
    Insurance_idInsurance = int(insurances[i])
    ChildSeat = int(np.random.choice([0, 1]))
    data.append((idCar, Lisense, Mark, Model, Insurance_idInsurance, ChildSeat))
    # print(data[i])
    
print("Q: ", query)
print("D: ", data[0])

#insert_many_rows(table, query, data)

## Table `driver` 

*** Changes ***
~~~~python
# Add isDeleted to check if driver working or not
cursor.execute("ALTER TABLE " + table + " ADD COLUMN isDeleted INT NOT NULL")
~~~~

In [ ]:
table = DB_NAME + '.' + 'driver' 
get_table_review(table)
count_rows(table)

In [ ]:
delete_all_rows(table)

In [ ]:
total_number = 75
query = "INSERT INTO TaxiTest.driver (idDriver, Name, DriverLisence, Phone, isDeleted) VALUES (%s, %s, %s, %s, %s)"

# Get all possible names_surnames combinations
all_names_surnames = []
with codecs.open("AllCombinations.txt", 'r', encoding='utf-8') as f:
    for line in f:
        all_names_surnames.append(line[:-2])

# Generate UNIQUE driver licenses
licenses = set()
while len(licenses) != total_number:
    part1 = ''.join(random.choice(string.ascii_uppercase) for _ in range(7))
    part2 = ''.join(random.choice(string.digits) for _ in range(4))
    licenses.add(part1 + part2)
licenses = list(licenses)


# Get names
data = []
names = list(np.random.choice(all_names_surnames, size=total_number))
for i in range(total_number):
    idDriver = i + 1
    # Failed processing format-parameters; Python 'str_' cannot be converted to a MySQL type
    Name = str(names[i])
    DriverLisence = licenses[i]
    Phone = '+7({}){:07}'.format(np.random.randint(50, 990), np.random.randint(0, 9999999))
    isDeleted = int(np.random.choice([0, 1], p=[0.15, 0.85]))
    data.append((idDriver, Name, DriverLisence, Phone, isDeleted))
    
print("Q: ", query)
print("D: ", data[0])

insert_many_rows(table, query, data)

## Table `address`

*** Changes ***
~~~~python
# Drop GeoPoint
cursor.execute("ALTER TABLE " + table + " DROP COLUMN GeoPoint
# Delete index on UNIQUE city
cursor.execute("ALTER TABLE " + table + " DROP INDEX City_UNIQUE")
~~~~

In [ ]:
table = DB_NAME + '.' + 'address' 
get_table_review(table)
count_rows(table)

In [ ]:
delete_all_rows(table)

In [ ]:
total_number = 1000
query = "INSERT INTO TaxiTest.address (idAddress, City, Street, HouseNumber) VALUES (%s, %s, %s, %s)"

City = "Kazan"
# Get all streets
with open('Streets.json') as f:
    streets = json.load(f)
    streets = np.random.choice(streets, total_number, replace=False)

# Get names
data = []
for i in range(total_number):
    idAddress = i + 1
    Street = str(streets[i])
    HouseNumber = int(random.randint(1, 100))
    data.append((idAddress, City, Street, HouseNumber))
    
print("Q: ", query)
print("D: ", data[0])

insert_many_rows(table, query, data)

## Table `taxi`

In [ ]:
table = DB_NAME + '.' + 'Driver'
get_table_review(table, True)
table = DB_NAME + '.' + 'Car'
get_table_review(table, True)
table = DB_NAME + '.' + 'Taxi'
get_table_review(table, True)

In [ ]:
# Try to select values from tables
table = "Address"
address_rows_amount = count_rows(table) # SQL script function

table = "Taxi"
# Using Taxi table attributes order to create query.
# INSERT INTO Taxi (idTaxi, Driver_idDriver, Car_idCar, Current_idAddress, Available, LatUpdate)
q = "SELECT idDriver, idCar, ROUND(RAND() * %s), ROUND(RAND()) FROM Driver, Car" % (address_rows_amount)

result = select_rows(table, q)
for row in result[:5]:
    print(row)

